In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

## Read Path

In [2]:
template_path = []

for template in os.listdir('PlateTemplates/template/'):
    template_path.append(os.path.join('PlateTemplates/template/', template))

In [3]:
template_path

['PlateTemplates/template/3.png',
 'PlateTemplates/template/4.png',
 'PlateTemplates/template/2.png',
 'PlateTemplates/template/1.png']

In [4]:
word_path = []

for word in os.listdir('PlateTemplates/word2/'):
    word_path.append(os.path.join('PlateTemplates/word2/', word))

In [5]:
number_path = []

for number in os.listdir('PlateTemplates/number2/'):
    number_path.append(os.path.join('PlateTemplates/number2/', number))

## Define Function For Agument Data

In [6]:
def ResizeImage(image):
    output_image = cv2.resize(image, (256, 65))
    return output_image

In [7]:
def zoom_in_out(image):
    height = 65
    width = 256
    
    height, width = image.shape[:2]
    
    zoom_factor = random.uniform(1.0, 1.08)
    new_height = int(height / zoom_factor)
    new_width = int(width / zoom_factor)
    
    top_left_y = random.randint(0, height - new_height)
    top_left_x = random.randint(0, width - new_width)
    
    image = image[top_left_y:top_left_y + new_height, top_left_x:top_left_x + new_width]
    
    image = cv2.resize(image, (256, 65))
    return image


In [8]:
import os
def MakeNoise(image):
    mean=0
    var=3
    amount=0.05
    salt_vs_pepper=0.2
    noise = np.random.randn(*image.shape) * 5
    for i in range(0, 50):
        image[random.randint(0, 64), random.randint(0, 255), :] = np.array((random.randint(230, 255), random.randint(230, 255), random.randint(230, 255)))

    for i in range(0, 50):
        image[random.randint(0, 64), random.randint(0, 255), :] = np.array((random.randint(60, 100), random.randint(60, 100), random.randint(60, 100)))

    if random.randint(0, 5) != 1:
        image = np.clip(image + noise, 0, 255).astype(np.uint8)





    
    noise_dir = 'Noises'
    
    if random.randint(0, 5) != 1:
        noise_image = cv2.imread('Noises/' + os.listdir('Noises')[random.randint(0, 7)])
        noise_image = ResizeImage(noise_image)


        
        if random.randint(0, 2) == 0:
            noise_image = cv2.rotate(noise_image, cv2.ROTATE_180)
        if random.randint(0, 2) == 0:
            noise_image = cv2.flip(noise_image, 0)
        if random.randint(0, 2) == 0:
            noise_image = cv2.flip(noise_image, 1)

        noise_image = ResizeImage(noise_image)



        
        image = ResizeImage(image)
        rand_num = random.randint(0, 2)
        image = cv2.add(image, noise_image)

    
    image = cv2.GaussianBlur(image, ksize=(15, 15), sigmaX=random.uniform(0.1, 2.5), sigmaY=random.uniform(0.1, 2.5))

    image = zoom_in_out(image)
    return image


In [9]:
def RandomColorAndBrightness(image):
    brightness = np.random.uniform(0.5, 2.0)
    contrast = np.random.uniform(0.9, 1.1)
    saturation = np.random.uniform(0.9, 1.1)
    
    output_image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    output_image_hsv[:,:,2] = np.clip(output_image_hsv[:,:,2] * brightness, 0, 255)
    output_image_hsv[:,:,1] = np.clip(output_image_hsv[:,:,1] * contrast, 0, 255)
    output_image_hsv[:,:,0] = np.clip(output_image_hsv[:,:,0] * saturation, 0, 255)
    
    output_image_color_augmented = cv2.cvtColor(output_image_hsv, cv2.COLOR_HSV2BGR)
    return output_image_color_augmented


def RandomColorAndBrightnessForAlef(image):
    brightness = np.random.uniform(0.5, 2.0)
    contrast = np.random.uniform(0.9, 1.1)
    
    output_image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    output_image_hsv[:,:,2] = np.clip(output_image_hsv[:,:,2] * brightness, 0, 255)
    output_image_hsv[:,:,1] = np.clip(output_image_hsv[:,:,1] * contrast, 0, 255)
    output_image_hsv[:,:,0] = np.clip(output_image_hsv[:,:,0] , 0, 255)
    
    output_image_color_augmented = cv2.cvtColor(output_image_hsv, cv2.COLOR_HSV2BGR)
    return output_image_color_augmented

In [10]:
def RandomRotation(image):
    angle = np.random.uniform(-3, 3)
    
    center = (image.shape[1] // 2, image.shape[0] // 2)
    
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    output_image_rotated = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    return output_image_rotated

# Police Template

In [11]:
def PreNumberPolice(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordPolice(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[378:635, 353:647]
    resized = cv2.resize(cropped_image, (240, 190))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w
    
def PoliceGenerator(count):
    if not os.path.exists("newData/Police/"): 
        os.makedirs("newData/Police/")
    temp_path = template_path[2]
    
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]

    for i in range(count):
        plate_name = 'newData/Police/'
        padded_img2 = np.zeros_like(template)

        p_word_preprocessed, logo_h_word, logo_w_word = PreWordPolice(word_path[0])
        location_h = 110
        location_w = 459
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPolice(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPolice(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'P'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPolice(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPolice(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPolice(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])
        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'

        output_image = cv2.add(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)

# white template

In [12]:
def PreNumberWhiteGeneratorMalol(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordWhiteGeneratorMalol(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:655, 353:647]
    resized = cv2.resize(cropped_image, (240, 220))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def WhiteGeneratorMalol(count):
    if not os.path.exists("newData/Paralyzed/"): 
        os.makedirs("newData/Paralyzed/")
    temp_path = template_path[0]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]

    for i in range(count):
        plate_name = 'newData/Paralyzed/'
        padded_img2 = np.zeros_like(template)

        p_word_preprocessed, logo_h_word, logo_w_word = PreWordWhiteGeneratorMalol(word_path[12])
        location_h = 55
        location_w = 465
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberWhiteGeneratorMalol(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberWhiteGeneratorMalol(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'F'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberWhiteGeneratorMalol(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberWhiteGeneratorMalol(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberWhiteGeneratorMalol(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY)

        output_image = cv2.bitwise_xor(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


# Taxi Template

In [13]:
def PreNumberTaxi(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordTaxi(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:635, 353:647]
    resized = cv2.resize(cropped_image, (240, 230))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def TaxiGenerator(count):
    if not os.path.exists("newData/Taxi/"): 
        os.makedirs("newData/Taxi/")
    temp_path = template_path[3]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]

    for i in range(count):
        plate_name = 'newData/Taxi/'
        padded_img2 = np.zeros_like(template)

        p_word_preprocessed, logo_h_word, logo_w_word = PreWordTaxi(word_path[6])
        location_h = 50
        location_w = 459
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberTaxi(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberTaxi(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'T'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberTaxi(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberTaxi(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberTaxi(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY_INV)

        output_image = cv2.bitwise_and(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


# Alef Template

In [14]:
def PreNumberAlef(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    resized = cv2.resize(cropped_number, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w

def PreWordAlef(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:635, 353:647]
    resized = cv2.resize(cropped_image, (220, 200))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w

def AlefGenerator(count):
    if not os.path.exists("newData/Alef/"): 
        os.makedirs("newData/Alef/")
    temp_path = template_path[1]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]

    for i in range(count):
        plate_name = 'newData/Alef/'
        padded_img2 = np.zeros_like(template)

        p_word_preprocessed, logo_h_word, logo_w_word = PreWordAlef(word_path[19])
        location_h = 80
        location_w = 470
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberAlef(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberAlef(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'A'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberAlef(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberAlef(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberAlef(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'

        output_image = cv2.add(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightnessForAlef(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


## Public

In [15]:
def PreNumberPublic(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w

def PreWordPublic(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:635, 353:647]
    resized = cv2.resize(cropped_image, (240, 230))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w

def PublicGenerator(count):
    if not os.path.exists("newData/Public/"): 
        os.makedirs("newData/Public/")
    temp_path = str(template_path[3])
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]

    for i in range(count):
        plate_name = 'newData/Public/'
        padded_img2 = np.zeros_like(template)

        p_word_preprocessed, logo_h_word, logo_w_word = PreWordPublic(word_path[18])
        location_h = 50
        location_w = 459
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPublic(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPublic(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'Z'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPublic(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPublic(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberPublic(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY_INV)

        output_image = cv2.bitwise_and(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


## Normal Plate License

In [16]:
def PreNumberXCN(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordXCN(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:655, 330:680]
    resized = cv2.resize(cropped_image, (230, 210))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def ParalyzedGeneratorXCN(count):
    if not os.path.exists("newData/Normal/"): 
        os.makedirs("newData/Normal/")
    temp_path = template_path[0]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]
    words_list = [30, 4, 23, 13, 24, 5]

    for i in range(count):
        plate_name = 'newData/Normal/'
        padded_img2 = np.zeros_like(template)
        rand_path_number = random.randint(0, 5)
        p_word_preprocessed, logo_h_word, logo_w_word = PreWordXCN(word_path[words_list[rand_path_number]])
        location_h = 75
        location_w = 465
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        temp = rand_path_number
        
        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberXCN(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberXCN(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        if temp == 0:
            plate_name = plate_name + 'X'
        elif temp == 1:
            plate_name = plate_name + 'C'
        elif temp == 2:
            plate_name = plate_name + 'N'
        elif temp == 3:
            plate_name = plate_name + 'Y'
        elif temp == 4:
            plate_name = plate_name + 'S'
        else:
            plate_name = plate_name + 'J'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberXCN(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberXCN(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberXCN(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY)

        output_image = cv2.bitwise_xor(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


In [17]:
def PreNumberDVM(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordDVM(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[355:655, 330:680]
    resized = cv2.resize(cropped_image, (260, 240))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def ParalyzedGeneratorDVM(count):
    if not os.path.exists("newData/Normal/"): 
        os.makedirs("newData/Normal/")
    temp_path = template_path[0]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]
    words_list = [16, 26, 21]

    for i in range(count):
        plate_name = 'newData/Normal/'
        padded_img2 = np.zeros_like(template)
        rand_path_number = random.randint(0, 2)
        p_word_preprocessed, logo_h_word, logo_w_word = PreWordDVM(word_path[words_list[rand_path_number]])
        location_h = 75
        location_w = 465
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        temp = rand_path_number


        
        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberDVM(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberDVM(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])
        
        if temp == 0:
            plate_name = plate_name + 'D'
        elif temp == 1:
            plate_name = plate_name + 'V'
        else:
            plate_name = plate_name + 'M'
        
        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberDVM(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberDVM(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberDVM(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY)

        output_image = cv2.bitwise_xor(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        cv2.imwrite(plate_name, output_image)

In [18]:
def PreNumberLK(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordLK(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[340:670, 330:680]
    resized = cv2.resize(cropped_image, (250, 240))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def ParalyzedGeneratorLK(count):
    if not os.path.exists("newData/Normal/"): 
        os.makedirs("newData/Normal/")
    temp_path = template_path[0]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]
    words_list = [11, 32]

    for i in range(count):
        plate_name = 'newData/Normal/'
        padded_img2 = np.zeros_like(template)
        rand_path_number = random.randint(0, 1)
        p_word_preprocessed, logo_h_word, logo_w_word = PreWordLK(word_path[words_list[rand_path_number]])
        location_h = 50
        location_w = 465
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        temp = rand_path_number
        
        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberLK(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberLK(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])
        if temp == 0:
            plate_name = plate_name + 'L'
        else:
            plate_name = plate_name + 'K'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberLK(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberLK(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberLK(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY)

        output_image = cv2.bitwise_xor(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)


In [19]:
def PreNumberB(path):
    number_path = path
    number = cv2.imread(number_path)
    cropped_number = number[350:593, 422:567]
    dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
    resized = cv2.resize(dilation, (145, 260))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w


def PreWordB(path):
    p_path = path
    word = cv2.imread(p_path)
    cropped_image = word[340:670, 320:690]
    resized = cv2.resize(cropped_image, (280, 210))
    ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
    logo_h, logo_w, _ = resized.shape
    return resized, logo_h, logo_w



def ParalyzedGeneratorB(count):
    if not os.path.exists("newData/Normal/"): 
        os.makedirs("newData/Normal/")
    temp_path = template_path[0]
    temp = cv2.imread(temp_path)
    template = temp[45:390, 15:1553]
    words_list = [2]

    for i in range(count):
        plate_name = 'newData/Normal/'
        padded_img2 = np.zeros_like(template)
        rand_path_number = random.randint(0, 0)
        p_word_preprocessed, logo_h_word, logo_w_word = PreWordB(word_path[words_list[rand_path_number]])
        location_h = 90
        location_w = 465
        padded_img2[location_h:location_h+logo_h_word,location_w:location_w+logo_w_word,:] = p_word_preprocessed

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberB(number_path[rand_path_number])
        location_h = 45
        location_w = 165
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberB(number_path[rand_path_number])
        location_h = 45
        location_w = 315
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        plate_name = plate_name + 'B'

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberB(number_path[rand_path_number])
        location_h = 45
        location_w = 710
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberB(number_path[rand_path_number])
        location_h = 45
        location_w = 865
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        number_preprocessed, logo_h_number, logo_w_number = PreNumberB(number_path[rand_path_number])
        location_h = 45
        location_w = 1030
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = number_preprocessed
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:590, 410:570]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1225
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        rand_path_number = random.randint(0, 8)
        path = number_path[rand_path_number]
        number = cv2.imread(path)
        cropped_number = number[350:593, 422:567]
        dilation = cv2.dilate(cropped_number, np.ones((5,5), np.uint8), iterations = 1)
        resized = cv2.resize(dilation, (135, 235))
        ret, resized = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY_INV)
        logo_h_number, logo_w_number, _ = resized.shape
        location_h = 82
        location_w = 1360
        padded_img2[location_h:location_h+logo_h_number,location_w:location_w+logo_w_number,:] = resized
        plate_name = plate_name + str(number_path[rand_path_number].split('/')[-1].split('.')[0])

        
        random_number = random.randint(1000, 1000000)
        plate_name = plate_name + '_' + str(random_number) + '.jpg'
        ret, padded_img2 = cv2.threshold(padded_img2, 127, 255, cv2.THRESH_BINARY)

        output_image = cv2.bitwise_xor(template, padded_img2)
        output_image = output_image[...,::-1]
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

        output_image = ResizeImage(output_image)
        output_image = MakeNoise(output_image)
        output_image = RandomColorAndBrightness(output_image)
        output_image = RandomRotation(output_image)
        
        cv2.imwrite(plate_name, output_image)

# X = ق = 30
# C = س = 4
# B = ب = 2
# M = م = 21
# J = ج = 5
# N = ن = 23
# L = ل = 11
# Y = ی = 13
# D = د = 16
# H = ه = 3
# K = ط = 32
# S = ص = 24
# V = و = 26

In [20]:
def generate(counts):
    count = counts / 17
    PoliceGenerator(int(count)) # 1
    WhiteGeneratorMalol(int(count)) # 1
    TaxiGenerator(int(count)) # 1
    AlefGenerator(int(count)) # 1
    PublicGenerator(int(count))    
    ParalyzedGeneratorXCN(int(count) * 6) # 6
    ParalyzedGeneratorDVM(int(count) * 3) # 3
    ParalyzedGeneratorLK(int(count) * 2) # 2
    ParalyzedGeneratorB(int(count)) # 1

generate(2000)